In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time
import klib
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.optimizers import Adam

In [ ]:
#load processed data
df = pd.read_csv('KDDTrain+_cleaned.csv')

In [12]:
# train a cnn model
X = df.drop(['type','class'], axis=1)
y = df['type']
# one hot encoding
y = pd.get_dummies(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# convert to numpy array
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

model = Sequential()
model.add(Conv1D(64, 3,activation="relu",input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(64, 3, activation="relu"))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(5, activation="softmax"))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

mc = ModelCheckpoint('cnn_best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

start = time.time()
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test), callbacks=[mc])
end = time.time()
print("Time taken to train the model: ", end-start)

Epoch 1/10
8402/8418 [============================>.] - ETA: 0s - loss: 0.0509 - accuracy: 0.9801
Epoch 1: val_accuracy improved from -inf to 0.98958, saving model to cnn_best_model.h5
8418/8418 [==============================] - 23s 3ms/step - loss: 0.0508 - accuracy: 0.9802 - val_loss: 0.0199 - val_accuracy: 0.9896
Epoch 2/10
  59/8418 [..............................] - ETA: 22s - loss: 0.0205 - accuracy: 0.9926

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8418/8418 [==============================] - ETA: 0s - loss: 0.0225 - accuracy: 0.9912
Epoch 2: val_accuracy improved from 0.98958 to 0.99385, saving model to cnn_best_model.h5
8418/8418 [==============================] - 22s 3ms/step - loss: 0.0225 - accuracy: 0.9912 - val_loss: 0.0145 - val_accuracy: 0.9939
Epoch 3/10
8404/8418 [============================>.] - ETA: 0s - loss: 0.0180 - accuracy: 0.9933
Epoch 3: val_accuracy improved from 0.99385 to 0.99448, saving model to cnn_best_model.h5
8418/8418 [==============================] - 22s 3ms/step - loss: 0.0181 - accuracy: 0.9933 - val_loss: 0.0129 - val_accuracy: 0.9945
Epoch 4/10
8409/8418 [============================>.] - ETA: 0s - loss: 0.0158 - accuracy: 0.9942
Epoch 4: val_accuracy improved from 0.99448 to 0.99516, saving model to cnn_best_model.h5
8418/8418 [==============================] - 22s 3ms/step - loss: 0.0158 - accuracy: 0.9942 - val_loss: 0.0134 - val_accuracy: 0.9952
Epoch 5/10
8408/8418 [=======================

In [13]:
model = load_model('cnn_best_model.h5')
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print("Time taken to predict: ", end-start)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

2105/2105 [==============================] - 2s 668us/step
Time taken to predict:  1.8207061290740967
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13340
           1       1.00      1.00      1.00     13507
           2       1.00      1.00      1.00     13499
           3       1.00      0.99      0.99     13629
           4       0.98      1.00      0.99     13368

    accuracy                           1.00     67343
   macro avg       1.00      1.00      1.00     67343
weighted avg       1.00      1.00      1.00     67343

0.9968073890382074


In [14]:
# from confusion matrix calculate accuracy of each class
cm = confusion_matrix(y_test, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Accuracy of each class:')
print('dos:', cm[0,0])
print('normal:', cm[1,1])
print('probe:', cm[2,2])
print('r2l:', cm[3,3])
print('u2r:', cm[4,4])

Accuracy of each class:
dos: 1.0
normal: 1.0
probe: 0.9991851248240611
r2l: 0.985031917235307
u2r: 1.0
